# Deployment
Saving and evaluating the fine-tuned model.

In [ ]:
# Save model and tokenizer
model.save_pretrained("t5_finetuned_summary")
tokenizer.save_pretrained("t5_finetuned_summary")

In [ ]:
# Load model and tokenizer (optional step for inference)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("t5_finetuned_summary")
tokenizer = AutoTokenizer.from_pretrained("t5_finetuned_summary")

In [ ]:
# ROUGE Evaluation
import evaluate
rouge = evaluate.load("rouge")

model.eval()
generated_summaries = []
true_summaries = []

for i in range(100):
    input_ids = small_dataset[i]["input_ids"].unsqueeze(0).to(model.device)
    attention_mask = small_dataset[i]["attention_mask"].unsqueeze(0).to(model.device)
    labels = small_dataset[i]["labels"]

    with torch.no_grad():
        output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)

    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    reference = tokenizer.decode(labels, skip_special_tokens=True)

    generated_summaries.append(generated)
    true_summaries.append(reference)

results = rouge.compute(predictions=generated_summaries, references=true_summaries)
print(results)